# Homework 6: Intro to Time Series
- **Course:** UAF GEOS604 &mdash; Seismology  
- **Instructor:** Bryant Chow ([bhchow@alaska.edu](bhchow@alaska.edu))
- **Course Website:** [https://bryantchow.com/teaching/geos604](https://bryantchow.com/teaching/geos604) 
- **Last Modified:**  02/24/25

## Semester: Spring 2025
- **Total Points**: 10
- **Assigned**: Not Yet Assigned
- **Due Date and Time**: 

---------
## Submitting Homework

1. If you have handwritten solutions to any problems:  
   a. include them directly in the notebook as a scan/picture in the appropriate section, OR  
   b. scan/photograph and include them separately. please also reference them in the notebook ("see handwritten notes submitted separately")
2. Save your homework as a PDF file using `File` -> `Save and Export Notebook As` -> `PDF`
3. Double check that the output PDF contains all your code, text, and any images you have included
5. Upload your PDF to the class Google Drive (see course website above) before the **Due Date and Time** listed above.

**See the class syllabus section on "Homework Policy" for late policy and information on academic integrity.**

---------------


## Homework Motivation
This homework will introduce key concepts and libraries used by research and operational seismologists. We will rely heavily on [ObsPy](https://docs.obspy.org/), an open-source Python package for handling seismic data and metadata. ObsPy has become a standard tool for modern-day seismologists. In this homework we will cover the following topics:  

1. Seismic Data and Metadata
2. Differences between data vs. metadata
3. Visualizing time series
4. Fourier analysis 
5. Instrument response removal
6. Preprocessing seismic data: detrend, taper, rotate, filter
7. Spectrograms

In [ ]:
# IMPORT CELL - Please run this cell before proceeding
import numpy as np
import matplotlib.pyplot as plt
import obspy

# These are imported again in cell blocks below, but referenced here for convenience
from obspy import read, read_events, read_inventory, UTCDateTime
from obspy.clients.fdsn import Client
from obspy.clients.nrl import NRL
from scipy.fftpack import fft, fftfreq, next_fast_len

# Custom plotting routine for ObsPy time series
from geos604funcs import plot_waveforms

%matplotlib widget

------------

# Problem 1: Seismic Data and Metadata [1.0]

- **Seismic data** are time series, made up of time stamps (absolute or relative) and data.
- Seismic data is numeric-only, data are stored as integer or floating point values. 
- **Seismic metadata** stores information about the station and seismic event associated with seismic data.
- Seismic metadata **must** be circulated with seismic data, so that seismologists know what their data describes.
- **ObsPy** is an open-source Python package that, among many other things, provides frameworks for handling seismic data and metadata.

### 1B: Stream

Seismic Data are stored in [Stream](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html) objects. The [read](https://docs.obspy.org/packages/autogen/obspy.core.stream.read.html) function can handle multiple formats of seismic data


In [ ]:
from obspy import read

st = read()  # loads example data
print(st)

> **Question**: ObsPy `Stream` objects can contain multiple `Trace` objects. In the `Stream` above ('st'), what is the difference between each of the `Trace`s?
> 
> **Answer**: 

We can access each [Trace](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.html) using Python's index notation. The [Trace.times()](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.times.html#obspy.core.trace.Trace.times) function and `Trace.data` attribute provide us the requisite parts of the time series data.

In [ ]:
# Print out the first 10 entries of the time series
print("Time\t  Data")
for t, d in zip(st[0].times()[:11], st[0].data[:11]):
    print(f"{t:.3f}\t  {d:1.4f}")
print("...\t  ...")

> **Question**: What are the units of `Time` and `Data` for the time series above?
> 
> **Answer**: 

> **Task:** The `times()` function can define the time array in a number of ways. Referencing the [`Trace.times()` documentation page](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.times.html#obspy.core.trace.Trace.times), print out a time array in the empty code cell below, formatted in the [UTCDatetime](https://docs.obspy.org/packages/autogen/obspy.core.utcdatetime.UTCDateTime.html#obspy.core.utcdatetime.UTCDateTime) format.

In [ ]:
# Task

ObsPy takes care of associating **metadata** with seismic data. The [Trace.stats](https://docs.obspy.org/packages/autogen/obspy.core.trace.Stats.html) attribute provides information on the seismic station that these data are associated to.

In [ ]:
tr = st[0]
print(tr.stats)

> **Question**: Using the FDSN station search (https://www.fdsn.org/networks/) determine 1) **where** (lat/lon) the seismic station is, 2) and who the operator is (Note that ObsPy started and is maintained by researchers at this university).
> 
> **Answer**: 

### 1C: Inventory

- The example data already comes with metadata attached in the `Trace.stats` attribute.
- Metadata are also stored in `Inventory` objects that define information for a given geophysical sensor.
- Similar to the `Stream` object, ObsPy `Inventory` objects contain multiple sub-categories for: networks, stations and channels.
- You can access each subsequent category of the `Inventory` using Python index notation

In [ ]:
from obspy import read_inventory

inv = read_inventory()
print(inv)  # Inventory
print(inv[0])  # Network
print(inv[0][0])  # Station
print(inv[0][0][0])  # Channel

>**Question:** The seismological community got together and defined the *Standard for the Exchange of Earthquake Data* (SEED) format for sharing seismic data and metadata. Channel codes are standardized to provide information about the recording instrument. Go to the [following informational page](http://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming/) which defines SEED channel naming. For station `FUR` (listed above), describe what each of the different letters in the `Available Channels` represents (you only have to define each unique letter once).
>
>**Answer:**

### 1D: Catalog

- **Event metadata** describes seismic sources (primarily earthquakes) that may give rise to seismic waveforms. 
- Event metadata is not always tied to seismic data since they do not always contain earthquake information
- So called **seismic noise** tends to be a mix of environmental and cultural sources, as well as other non-earthquake sources.
- The ObsPy [Catalog](https://docs.obspy.org/packages/autogen/obspy.core.event.Catalog.html) object, and corresponding `Event` object, are used to store event metadata
- Each event contains information about the earthquake `Origin` (hypocenter and origin time), `Magnitude`, and additional information (e.g., phase picks, focal mechanisms)

In [ ]:
from obspy import read_events

cat = read_events()
print(cat)  # Catalog
print(cat[0])  # Event 
print(cat[0].preferred_origin())  # Origin information
print(cat[0].preferred_magnitude())  # Magnitude information

> **Question:** What are the hypocenter (coordinates and depth), origin time, and magnitude of the **first** earthquake in the Catalog?
>
> **Answer:** 

-----------
# Problem 2: Fetching Seismic Data and Metadata [1.0]

### 2A: Background
- Seismic data from long-term seismic stations can be queried from public web services.
- [EarthScope](https://www.earthscope.org/about/earthscope/) (formerly IRIS, Incorporated Research Institutions for Seismology) is an federally funded consortium dedicated to stewarding geophysical facilities supporting long-term, fair and open access to geophysical instrumentation, observations, and practices.
- Thanks to EarthScope, and the efforts of seismologists around the world, a significant amount of seismic data are publicly open, freely accesible, and relatively straightforward to obtain.
- Seismic data and metadata are primarily made accesible through the [International Federation of Digital Seismograph Networks (FDSN)](https://www.fdsn.org/about/)

### 2B: Client
ObsPy has a built in [Client](https://docs.obspy.org/packages/obspy.clients.fdsn.html) module that allows us to query FDSN services for seismic data from a number of hosting institutions.


In [ ]:
from obspy.clients.fdsn import Client

client = Client("IRIS")
print(client)

The `Client` module has three main functions that we will use to query seismic data and metadata. ObsPy `Client` allows [Unix wildcards](https://tldp.org/LDP/GNU-Linux-Tools-Summary/html/x11655.htm). You may use wildcards to select for multiple components, stations, etc.

- [Client.get_waveforms()](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_waveforms.html#obspy.clients.fdsn.client.Client.get_waveforms): Download seismic waveform data
- [Client.get_events()](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_events.html#obspy.clients.fdsn.client.Client.get_events): Download seismic event metadata
- [Client.get_stations()](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_stations.html#obspy.clients.fdsn.client.Client.get_stations): Download seismic station metadata


### 2C: Get Event

> **Task**: Below you are provided unfinished code which will be used to query seismic data for the [2024 Hualien earthquake (Taiwan)](https://en.wikipedia.org/wiki/2024_Hualien_earthquake), recorded at station [IU.COLA](https://earthquake.usgs.gov/monitoring/operations/stations/IU/COLA/) in Fairbanks, AK.
>
> 1. Determine the origintime, magnitude and location of the 2024 Hualien earthquake, use `get_events()` to query a `Catalog` object containing event metadata.  
> 2. To find the earthquake, we must provide approximate magnitude and location, `get_events()` will search available events that match these criteria.  
> 3. If you get an `FDSNNoDataException` then double check your input values, something is incorrect.    
> 4. Make sure you are only returned 1 earthquake, if you have more than one, then you need to refine your inputs.  

In [ ]:
# Task 
from obspy import UTCDateTime

starttime = "" # YOUR INPUT HERE: Earthquake origin time, e.g., '2000-01-01T00:00:00'
minimum_magnitude =   # YOUR INPUT HERE
maximum_magnitude =   # YOUR INPUT HERE
latitude =   # YOUR INPUT HERE
longitude =   # YOUR INPUT HERE

# vvv Do not modify below vvv
cat = client.get_events(
    minmagnitude= minimum_magnitude,
    maxmagnitude=maximum_magnitude,
    latitude=latitude, 
    longitude=longitude, 
    maxradius=0.1,  # search radius in degrees
    starttime=UTCDateTime(starttime) - 60 * 5,  # Start looking a few minutes before origintime
    endtime=UTCDateTime(starttime) + 60 * 5,  # Stop looking a few minutes after origintime
)

# Print and plot to confirm that we selected the right event
print(cat)
print(cat[0])
cat.plot(projection="ortho")

> **Question:** Does your recovered Event match the information you queried exactly? (origin time, magnitude, latitude, longitude). If not, discuss why these might be different. 
>
> **Answer:**
>
 

### 2D: Get Stations

> **Task:**  Determine the network, station, and channel codes associated with [IU.COLA](https://earthquake.usgs.gov/monitoring/operations/stations/IU/COLA/). Query station metadata in an `Inventory` object.
> 1. There are two seismometers at station `IU.COLA`, a surface sensor and a borehole sensor. Query data only for the **borehole sensor**
> 2. Select your [channel code](http://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming/) such that we query data for a **Broad Band**, **High Gain Seismometer** for **all three components**
> 3. Print all 3 levels of your `Inventory` object to ensure you queried the correct metadata

In [ ]:
# Task
network_code  =   # YOUR INPUT HERE
station_code  =   # YOUR INPUT HERE
location_code =   # YOUR INPUT HERE
channel_code  =   # YOUR INPUT HERE

# vvv Do not modify below vvv
inv_cola = client.get_stations(
    network=network_code
    station=station_code, 
    location=location_code,  
    channel=channel_code,
    starttime = UTCDateTime(starttime) - 60 * 5,  # Start looking a few minutes before origintime
    endtime = UTCDateTime(starttime) + 60 * 5,  # Stop looking a few minutes after origintime
    level="response",  # Include instrument response (see later sections)
)
# ^^^ Do not modify above ^^^

# Print all levels 
# YOUR INPUT HERE

> **Question:** From the metadata you just collected, what is the exact depth (in meters) of the borehole seismometer at IU.COLA?
>
> **Answer:**
>
> **Question:** What are the three orthogonal components defining the orientation of the sensor. Why do you think the horizontal components are not listed as East and North? (hint: look at the channel metadata)
>
> **Answer:**



### 2E: Get Waveforms
> **Task:** Query seismic time series data for the exact station and components you queried above
> - Set your `starttime` as the origin time of the Hualien earthquake,
> - Set your `endtime` as 1.5 hours after the earthquake origintime
> - Note that you can either manually input these, or you can add seconds to a `UTCDateTime` object
```python
# Example adding 60 seconds to a UTCDateTime object 
UTCDatetime("2000-01-01T00:00:00") + 60  # = UTCDatetime("2000-01-01T00:00:60")
```

In [ ]:
# Task
st_cola = client.get_waveforms(
    network =  , # YOUR INPUT HERE
    station = ,  # YOUR INPUT HERE
    location = ,  # YOUR INPUT HERE
    channel = ,  # YOUR INPUT HERE
    starttime =  ,  # YOUR INPUT HERE
    endtime = # YOUR INPUT HERE
)

print(st_cola)

-----------

# Problem 3: Plotting a time series [1.0]

- ObsPy has a built-in [Stream.plot()](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.plot.html#obspy.core.stream.Stream.plot) function that allows you to quickly look at data, but their visualization is limited in scope in this Jupyter environment
- I have written a generic waveform plotting function `plot_waveforms()` so that we can have a more useful way to look at the waveform data.
- `plot_waveforms` takes your Stream object `st` as input. It also takes an input `fmt` which dictates how the x-axis deals with time, 'absolute' or 'relative'

In [ ]:
from geos604funcs import plot_waveforms

plot_waveforms(st_cola)

> **Question:** Approximately how many seconds after the event origin time (t=0) does the P-wave arrive?
>
> **Answer:**
>
> **Question:** Approximately how many seconds after the event origin time (t=0) does the S-wave arrive?
>
> **Answer:**
>
> **Question:** At approximately what time after the origin time (in seconds) does the surface wave train start and stop?
>
> **Answer:**
>
> 

> **Task:** Using your P and S picks:
> - Consult the travel time chart below which shows travel time curves for body wave phases for model IASP91
> - Determine an approximate distance, in units of degrees, between the earthquake in Taiwan, and Fairbanks, AK. 
> - Compare your answer by measuring the distance in Google Maps. How accurate was your phase arrival estimation?
>

<img src="https://levee.wustl.edu/seismology/book/chapter3/chap3_fr/3_5_04.jpg" width=900 height=900 class="center"/>


------------

# Problem 4: Reading Local Data [1.0]

### 4A: Background
- Seismologists who perform their own field deployments need to access their instruments' data stored on local harddrives or databases.
- Seismic instruments output their data in a variety of different formats. Often a field seismologist's first task after deployment is to get their data into a usable format.
- Obspy's [read() function](https://docs.obspy.org/packages/autogen/obspy.core.stream.read.html) includes a number of built-in formats for reading seismic data in as `Stream` objects

### 4B: Read Data
- I have hosted 3 waveform files for a seismic instrument (node) deployed in 2024
- Course Google Drive Link: https://drive.google.com/drive/u/0/folders/1SKpSB2cLfJdLGQisZwt9Z8tthBQu_0ZK
- This instrument was deployed just outside the [Large Animal Research center (LARS)](https://www.google.com/maps/place/Large+Animal+Research+Station/@64.8783669,-147.8690703,674m/data=!3m2!1e3!4b1!4m6!3m5!1s0x5132435a0ab440d1:0x7074e2c00615af0d!8m2!3d64.8783669!4d-147.86649!16s%2Fg%2F1v16pyqr?entry=ttu&g_ep=EgoyMDI1MDIxOS4xIKXMDSoJLDEwMjExNDU1SAFQAw%3D%3D).   
- Fortunately the time span of this deployment captured the Hualien earthquake
  
> **Task:** Read, manipulate and visualize this data
> 1. **Download** the waveform files and make them accessible to your notebook.
> 2. **Read** in all 3 available data files stored in `./data` in to **one** Stream object called `st_node` (Note that you can add `Stream` objects together, e.g., `stream = stream_a + stream_b + stream_c`)
> 3. **Trim** the `Stream` object to match the `starttime` and `endtime` of IU.COLA shown above (https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.trim.html) 
> 4. **Plot** the trimmed `Stream` object to show the waveforms using `plot_waveforms()`


In [ ]:
# Task 1 - Read data
st_node =   # YOUR INPUT HERE
# st_node += read(...)  # <- Example code for adding two streams together

# Task 2 - Trim data
starttime =  # YOUR INPUT HERE
endtime =    # YOUR INPUT HERE
st_node.trim(starttime=starttime, endtime=endtime)

# Task 3 - Plot waveform

> **Question:** The Hualien earthquake is not obvious on the node data. So what do you think this instrument is recording? Consider the geographic location of the site and the potential sources of seismic energy nearby. Consider the duration and frequency for the signals you see.
>
> **Answer:**
>
> **Question:** Speculate on why you think this instrument, which is only half a kilometer from IU.COLA (at CIGO), has such a different recording for the same time period, compared to IU.COLA
>
> **Answer:**

--------

# Problem 5: Fourier Analysis [0.5]

- **Fourier analysis** is the study of the way general functions may be represented or approximated by sums of simpler trigonometric functions.
- **Fourier transforms** can be used to **decompose** continuous signals (e.g., seismogram) into a summation of trigonemtric functions, each with a single frequency.
- The [**Fast Fourier Transform (FFT)**](https://en.wikipedia.org/wiki/Fast_Fourier_transform) is a computer algorithm for very efficiently computing fourier transforms for discrete time series.


>**Task:** Run the following code block which uses the [SciPy](https://scipy.org/) implementation of the [Fast Fourier Transform](https://docs.scipy.org/doc/scipy/tutorial/fft.html) to decompose our time series for IU.COLA

In [ ]:
from scipy.fftpack import fft, fftfreq, next_fast_len

# Calculate FFT of IU.COLA data for all three components
plt.close("all")
f, ax = plt.subplots()
for tr in st:
    sr = tr.stats.sampling_rate
    nfft = next_fast_len(tr.stats.npts)  # pad data with zeros for fast FT
    pos_freq = (nfft + 1) // 2  # index for positive frequencies
    
    spec = fft(tr.data, nfft)[:pos_freq]  # calculate the FFT
    freq = fftfreq(nfft, 1 / sr)[:pos_freq]  # get freqs of DFT bin
    plt.plot(freq, np.abs(spec), lw=1, label=tr.get_id(), alpha=0.5)  

plt.legend(frameon=False)
plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude")
plt.loglog()
plt.grid()
plt.xlim([1E-5, 10]) # units Hz

> **Question:** Describe what you see in the figure above, what are the two axes, and what does the line shown represent?
>
> **Answer:**
>
> **Question:** In terms of period (1/frequency), where is most of the energy in this waveform? 
>
> **Answer:**
>
> **Question:** When performing FFT on a computer, we add zeros to our data to ensure it is a certain length. We did this above with the function `next_fast_len`. Read the [description of function `next_fast_len`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.next_fast_len.html) to determine why we perform this operation.
>
> **Answer:**

----------

# Problem 6: Instrument Response [1.0]

### 6A: Background
- Seismometers do not directly measure ground motion, but instead measure voltages induced by the movement of a mass suspended in a magnetic field.
- As a result, the raw instrument output of a seismometer is in units of **digital counts**.
- There are a number of dynamic systems in a seismometer that define an allowable range of ground motions and frequencies that an instrument is sensitive to.
- Each instrument therefore has a different **response** to the same input ground motion. Instrument response is unique to a specific make, model and calibration of a seismometer.
- The **Instrument Response** defines the transfer function that converts from digital counts to ground motion (i.e., displacement, velocity, acceleration). 
- Seismologists **remove instrument response** so that time series are in units of true ground motion, and do not include instrument effects.
- The [Nominal Response Library (NRL)](https://ds.iris.edu/ds/nrl/) is a database of expected instrument response defined by manufacturer specifications. 
- Note that seismologists often do careful calibrations to determine the exact response of specific instruments in their charge, rather than relying on "nominal" values.


### 6B: COLA Response

The borehole sensor at IU.COLA is a [Kinemetrics STS-6A](https://kinemetrics.com/promos/STS-6APROMO/index.html), an observatory-grade seismometer that is very stable over a large range and frequency of input motions.


In [ ]:
print(inv_cola)
print(inv_cola[0][0][0].response)

We can [plot the ObsPy `Response` object](https://docs.obspy.org/packages/autogen/obspy.core.inventory.response.Response.plot.html) to have a look at characteristics of the response curve.

In [ ]:
inv_cola[0][0][0].response.plot(output="VEL", min_freq=1E-4)

> **Question:** The response plot shows how input waveforms are modified in amplitude and phase based on their frequency. Instruments are said to have "flat response" where the amplitudes of frequencies are not attenuated by the instrument. For IU.COLA (shown above) define where the instrument has a flat response. Express your answer in period (units: seconds).
>
> **Answer:**
>
> **Question:** The vertical dashed line in the figures above shows the maximum frequency allowable by the seismometer. For COLA, how does this value compare to the sampling rate? What is this frequency called?
> 
> **Answer:**
>

### 6C: Node Response

- The instrument that Bryant deployed at LARS is a [Magseis-Fairfield Z-Land seismic node](https://epic.earthscope.org/content/instrumentation/all-one-systems/nodes). 
- Nodes are self-contained seismometers, which sacrifice stability for portability.
- Seismic nodes therefore have a very different response from high-quality stations like IU.COLA, as these instruments are intended for different applications.

Let's compare the node's response to COLA. We can query the NRL for response information.


In [ ]:
from obspy.clients.nrl import NRL

nrl = NRL()
node_response = nrl.get_response(
    sensor_keys = ["Magseis Fairfield", "Generation 2", "5 Hz"],
    datalogger_keys = ["Magseis Fairfield", "Zland 1C or 3C", "18 dB (8)", "250", "Linear Phase", "Off"]
)
node_response.plot(output="VEL", min_freq=1E-4)

> **Question:** How does the node response compare to COLA? Looking at the response differences, do you have a suspicion why we cannot see the Hualien earthquake in the raw node data?
>
> **Answer:**
>
> **Question:** Comparing the node response to IU.COLA, speculate on the differences in intended applications of nodes vs. an STS-6A

### 6D: Response Removal

We can **remove instrument response** using ObsPy. There are multiple ways of doing this, I'll show you one using the [remove_response() function](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.remove_response.html)

In [ ]:
st_cola_rr = st_cola.copy()  # Copy the data so that we retain raw data

# Remove response and plot the procedure
f, ax = plt.subplots(figsize=(9,8))
st_cola_rr.remove_response(inventory=inv_cola, output="VEL", plot=True, fig=f)

In [ ]:
plot_waveforms(st_cola_rr)

>**Question:** There are a number of figures shown for the response removal plot. Describe what is happening in each of the rows in this figure.
>
>**Answer:**
>
>**Question:** Now that we have removed instrument response, our data is in units of ground motion. What is the quantity and what are the units of the time series above?
>
>**Answer:**
>


----------

# Problem 7: Preprocessing seismic data [1.0]

### 7A: Background
Seismologists often have to complete additional **preprocessing** tasks which are meant to further "clean up" the waveforms prior to analysis. Typical preprocessing tasks include:
   - [Resample](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.resample.html): Changes the sampling rate of the data. Typical sampling rates are around 100 Hz, which may be too high for analyses. We tend to **downsample data** to make it easier to work with.
   - [Detrend](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.detrend.html): Detrending removes linear or polynomial trends from data. Seismologists may only consider a few hours of data at a time. Long period (>1 hour) trends are therefore not needed in the analysis, and can often obscure other parts of the waveform.   
   - [Taper](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.taper.html): Tapering "rounds off" the beginning and end of our signals. Seismic data are continuous, and signals prior to an earthquake origin time, or after the end of the surface wave train, may not be desired during analysis. Additionally, many of the signal processing techniques we perform require that data start and end at zero amplitude. 
   - [Rotate](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.rotate.html): Re-orient the seismometer geographically. Seismometers record on three othogonal components, and seismologists can "choose" how to orient these components in space. Common coordinate systems include North-South, East-West, and Vertical (ZNE), as well as Radial, Transverse, Vertical (RTZ). RTZ rotations are **event-dependent**, meaning the radial direction is in line with the great-circle path between event and receiver.


### 7B: Preprocessing
> **Task**: In an **order of your choosing**, apply the following preprocessing functions to the IU.COLA data. Confirm that your preprocessing was successful by ensuring that there are no long-period trends, that your data touches 0 at the start and end, and that your data are in the RTZ coordinate system.

> 
> - **Resample** the data to 10 samples per second
> - **Detrend** the data to get rid of the long-period trend
> - **Taper** your data so that both the start and end of the data are at 0, but that you do not affect any signal of interest, and do not introduce artefacts.
> - **Rotate** your three components to the RTZ coordinate system.
>     - You will first need to rotate from `12Z` to `ZNE`. This requires the `Inventory` you collected earlier.
>     - You will then have to rotate from `ZNE` to `RTZ` using the appropriate backazimuth.
>     - You can use the [gps2dist_azimuth](https://docs.obspy.org/packages/autogen/obspy.geodetics.base.gps2dist_azimuth.html) function to find the appropriate backazimuth.
>

**Important Note:** Each of these operations is performed **in-place**, i.e., your original data is lost. If you require the original unprocessed data, you will need to use the [Stream.copy() method](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.copy.html)

In [ ]:
from obspy.geodetics import gps2dist_azimuth

# Use a copy so that we can easily reset if we change processing
st_preproc = st_cola_rr.copy()

# Task 1: Use a combination of `resample`, `detrend`, and `taper` to remove long period trends
# <-- YOUR INPUT HERE 

# Task 2: Calculate Backazimuth
dist_m, az, baz = gps2dist_azimuth(
    lat1= ,  # <-- YOUR INPUT HERE 
    lon1= ,  # <-- YOUR INPUT HERE 
    lat2= ,  # <-- YOUR INPUT HERE 
    lon2= ,  # <-- YOUR INPUT HERE 
)
print(f"Distance={dist_m*1E-3:.2f}km\n"
      f"Azimuth={az:.2f}deg\n"
      f"Backazimuth={baz:.2f}deg")
    
# Task 3: Rotate 12Z -> ZNE -> RTZ    
# <-- YOUR INPUT HERE 

# Task 4: Plot the waveforms against the unprocessed version to check how preprocessing affected data
plot_waveforms(st_preproc, st_2=st_cola_rr)

### 7C: Filters

- As we saw in the fourier analysis, our seismic data is a summation of energy arriving at different frequencies
- Seismologists use **filters** to isolate or remove certain frequencies, depending on the analysis they're doing
- There are three main filters we typically use:
    1. **Bandpass**: Filter out a specific range of frequencies, between $f_{min}$ and $f_{max}$
    2. **Lowpass**: Only allow (pass) low-frequency/long period energy, i.e., filter out high frequency/short period energy for a given $f_{cutoff}$
    4. **Highpass**: Only allow high-frequency/short period energy, i.e., filter out low frequency/long period energy for a given $f_{cutoff}$



> **Task**: Filter the IU.COLA data using the three types of filters
> 1. Apply a **bandpass** filter for periods between 20 and 30s
> 2. Apply a **highpass** filter for periods above 2s
> 3. Apply a **lowpass** filter for periods below 40s
> 4. **Plot** to result of each filter
>
> **Important Note:** Filtering is also performed **in-palce**. Ensure that you are copying the data each time so that you are not filtering the same data multiple times. 

In [ ]:
# Task 1
st_filter_bandpass = st_preproc.copy()

# <-- YOUR INPUT HERE 


In [ ]:
# Task 2
st_filter_highpass = st_preproc.copy()

# <-- YOUR INPUT HERE 


In [ ]:
# Task 3
st_filter_lowpass = st_preproc.copy()

# <-- YOUR INPUT HERE 

> **Question:** For the Hualien earthquake recorded at IU.COLA, are the S-waves primarily recorded as short period (< 1s) or long periods (> 1s)
>
> **Answer:**
>
> **Question:** What about the P-waves?
>
> **Answer:**
>
> **Question:** Surface waves are often the dominant signal on a seismogram, are these short or long-period energy?
>
> **Answer:**

---------

# Problem 8: Spectrograms [1.0]

### 8A: Background
- A spectrogram is a visual representation of the spectrum of frequencies of a signal as it varies with time.
- Spectrograms represent the distribution of energy over frequencies, for each time during a seismogram.
- Seismologists use spectrograms to understand time-dependent energy arrival and dynamics.
- ObsPy has a built in [Spectrogram](https://docs.obspy.org/packages/autogen/obspy.imaging.spectrogram.spectrogram.html) function 


### 8B: COLA Spectrogram
> **Task:** Plot a spectrogram for the vertical (Z) component of your processed IU.COLA data stored in `st_preproc`
> - You can use the [Stream.select()](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.select.html) function to deal with only one component.
> - Set `log=True` to get a logarithmic scale on the Y-axis


In [ ]:
# YOUR INPUT HERE

> **Question:** Describe what you see in the spectrogram. Make comparisons to the vertical component waveform and comment on what the colors mean.
>
> **Answer:**
>


--------

# Problem 9: Probablistic Power Spectral Densities [1.0]

- We saw a **Power Spectral Density** (PSD) in the previous section on Fourier transforms. The frequency-amplitude figure shows relative power of each spectra.
- Probablistic power spectral densities (PPSD) are one way of viewing many PSDs at once and finding empirical averages.
- PPSD's are typically used to determine noise levels of a station, by creating consecutive (hourly, daily, etc.) PSDs and finding their mean value
- ObsPy has a **built-in** PPSD function: https://docs.obspy.org/tutorial/code_snippets/probabilistic_power_spectral_density.html

> **Task**: Create a Probablistic Power Spectral Density for station IU.COLA
> 1. Download **vertical component** seismic data for the `IU.COLA` **borehole** sensor.
> 2. Ensure that you collect 12 hours of data on either side of the 2024 Hualien earthquake origin time (24 hours of total data)
> 3. Create a PPSD for IU.COLA during this time range
> 4. Plot the PPSD with the `pqlx` coloarmap (see ObsPy tutorial for how to apply this)

In [ ]:
# Task - Collect waveform data for IU.COLA

In [ ]:
# Task - Create PPSD

In [ ]:
# Task - Plot PPSD

> **Question:** How many segments are contained in this PPSD. Roughly how much time does each of these segments contain?
> 
> **Answer:**
>
> **Question:** What are the light gray lines above and below the PPSD? What do they represent and how were they calculated?
>
> **Answer:**
>
> **Question:** There are a few segments of the PPSD that have signficant amplitude differences with respect to the mean values. What is this coming from? Might we expect to see similar behavior for a PPSD of the next day of data?
>
> **Answer:**

-----------

# Problem 10: Processing Nodal Data [1.5]

We have performed basic signal processing on the IU.COLA data. Let's do the same but for the node data. You have all the pieces in this notebook that you need to complete the following tasks.

> **Task: Preprocess, filter and plot the node data**
> 1. **Remove instrument response** for the nodal data (the appropriate metadata can be found in the `data/` directory
> 2. **Preprocess** the waveforms in the same way we did IU.COLA (detrend, taper, rotate)
> 3. **Filter** the node data. Choose an appropriate filter and filter bounds to **isolate** the Hualien earthquake signal from all other energy (use IU.COLA as a reference)
> 4. **Plot** the processed and filtered node data, alongside the data from IU.COLA
> 5. Plot a **spectrogram** for the vertical component of the node data

In [ ]:
# Task 1: Remove instrument response
st_node_proc = st_node.copy()  # Copy node data so we don't overwrite raw data
# YOUR INPUT HERE

# Task 2: Preprocess Node data
# YOUR INPUT HERE

# Task 3: Filter node data
# YOUR INPUT HERE

# Task 4: Plot node data
# plot_waveforms(st_node_proc, st_2=st_preproc)  # <- Example code block to plot node data alongside IU.COLA

# Task 5: Plot node spectrogram

> **Question:** What are some major differences between the processed node data and IU.COLA?
>
> **Answer:**
>
> **Question:** The node data does not show surface wave data as well as IU.COLA, why is this?
>
> **Answer:**
>
> **Question:** Given the fact that this node was deployed near LARS, what do you think is contributing to additional seismic energy recorded on the node, as compared to IU.COLA?
>
> **Answer:**

-----------

## Problem N 
1. Approximately how much time did you spend on this homework assignment?
2. Did you find this homework particularly easy, adequate, or difficult?
3. Any feedback on the homework?
